In [ ]:
import os
import gc
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"] = "SimHei"
import seaborn as sns

In [ ]:
# 文件地址
filepath = os.getcwd() + '/data_sets/'
# 用户行为日志
user_log = pd.read_csv(filepath + 'user_log_format1.csv')
# 用户画像
user_info = pd.read_csv(filepath + 'user_info_format1.csv')
# 训练数据
train1 = pd.read_csv(filepath + 'train_format1.csv')
# 测试数据
test1 = pd.read_csv(filepath + 'test_format1.csv')

user_id: 购物者的唯一ID编码；
item_id: 商品的唯一编码；
cat_id: 商品所属品类的唯一编码；
merchant_id: 商家的唯一ID编码；
brand_id: 商品品牌的唯一编码；
time_tamp: 购买时间（格式：mmdd）；
action_type: 包含{0, 1, 2, 3}，0表示单击，1表示添加到购物车，2表示购买，3表示添加到收藏夹；

user_id: 购物者的唯一ID编码；
merchant_id: 商家的唯一ID编码；
label: 包含{0, 1}，1表示重复买家，0表示非重复买家。测试集这一部分需要预测，因此为空。

age_range：用户年龄范围，<18岁为1；[18,24]为2； [25,29]为3； [30,34]为4；[35,39]为5；[40,49]为6； >= 50 时为7和8; 0和NULL表示未知
	
gender：用户性别，0表示女性，1表示男性，2和NULL表示未知

In [ ]:
for df in [user_log, user_info, train1, test1]:
    print(df.shape)
    print(list(df.columns))
    
user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)

del df
gc.collect()

In [ ]:
user_info.info()
user_info.isnull().sum()

In [ ]:
user_info['gender1'] = user_info['gender'].replace({0:'女性', 1:'男性', 2:'未知'})
user_info['gender1'].fillna('空值',inplace=True)
gender = user_info['gender1'].value_counts().reset_index().rename(columns={'index':'gender', 'gender1':'counts'})

In [ ]:
ax = gender.plot.bar(x='gender', y='counts', rot=0) # pandas.DataFrame.plot.bar()

In [ ]:
# user_info['age_range'].value_counts()
user_info['age_range1'] = user_info['age_range'].replace({0:'未知', 1: '<18', 2: '[18,24]', 3: '[25,29]', 4: '[30,34]', 5: '[35,39]', 6: '[40,49]', 7 :'[50,59]', 8:'>=60'})
user_info['age_range1'].fillna('空值',inplace=True)
age_range = user_info['age_range1'].value_counts().reset_index().rename(columns={'index':'age_range', 'age_range1':'counts'})
age_range

In [ ]:
ax = age_range.plot.bar(x='age_range', y='counts', rot=0)

In [ ]:
user_info.head()
pd.crosstab(user_info.gender1, user_info.age_range1, margins=True)

In [ ]:
user_log.info()
user_log.isnull().sum()

In [ ]:
user_log['brand_id'].fillna(0, inplace=True)
user_log = user_log.apply(lambda col: col.astype('int32'))

In [ ]:
user_log.action_type.value_counts()
action_type = user_log.action_type.value_counts().reset_index().rename(columns={'index':'action', 'action_type':'type_count'})
ax = action_type.plot.bar(x='action', y='type_count', rot=0)

In [ ]:
train1['data_flag'] = 'train'
test1['data_flag'] = 'test'
matrix = pd.concat([train1, test1], ignore_index=True, sort=True)
matrix.drop(['prob'], axis=1, inplace=True)
matrix = matrix.merge(user_info, on='user_id', how='left')

del train1, test1, user_info
gc.collect() # 若被调用时不包含参数，则启动完全的垃圾回收